In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import pandas as pd
import numpy as np
from sensible_raw.loaders import loader
import preprocess_relation_helper as ph

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# set time range for the time series

CNS_TIME_BEGIN = pd.Timestamp(pd.datetime(2014, 1, 1)) # first timestamp 
CNS_TIME_END = pd.Timestamp(pd.datetime(2014, 4, 30)) # last timestamp

time = pd.DataFrame(pd.date_range(CNS_TIME_BEGIN, CNS_TIME_END, freq='d'),columns=['time'])

In [15]:
# load data
data = pd.DataFrame()
            
time_slices = ['january_2014', 'february_2014', 'march_2014', 'april_2014']
for t in time_slices:
    data_t = loader.load_data("bluetooth", t , as_dataframe=True) 
    data = pd.concat([ data, data_t])

In [16]:
# filter by signal strength, valid users, 
data = ph.Helper.filter_raw_data(data, CNS_TIME_BEGIN, CNS_TIME_END)

/home/lochnerpik@gmail.com/PreprocessRelations/preprocess_relation_helper.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  data = pd.concat([data, data_inv])


In [17]:
# calculate meetings of to nodes per day using gaussian smoothing
data = ph.Helper.agg_to_visits_per_day(data, time)

In [7]:
data.describe()

,id_A,id_B,encounter
count,1.714476e+07,1.714476e+07,1.714476e+07
mean,2.309464e+02,4.655297e+02,4.572542e-01
std,1.618127e+02,1.691337e+02,5.146374e+00
min,0.000000e+00,3.000000e+00,0.000000e+00
25%,9.500000e+01,3.490000e+02,0.000000e+00
50%,2.010000e+02,4.860000e+02,0.000000e+00
75%,3.480000e+02,5.990000e+02,0.000000e+00
max,7.970000e+02,8.450000e+02,5.400000e+02


In [8]:
data.loc[data.encounter <= 1, 'encounter'] = 0 # just check if there was an encounter
data.loc[data.encounter > 1, 'encounter'] = 1

In [11]:
data.rename(columns={"encounter":"edge"},inplace=True)
data[['id_A','id_B','time','edge']].to_pickle("../data/relations.pkl")